In [1]:
!nvidia-smi

Sun May  1 14:21:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install -Uqq transformers

     |████████████████████████████████| 4.0 MB 12.3 MB/s 
     |████████████████████████████████| 895 kB 46.6 MB/s 
     |████████████████████████████████| 77 kB 6.5 MB/s 
     |████████████████████████████████| 6.6 MB 20.6 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 


In [14]:
import pandas as pd
data_url = 'https://raw.githubusercontent.com/Statisticians-Without-Borders-GVCEH/SWB-GVCEH/main/scraper/data/'
fname = 'GVCEH-2022-04-27-tweet-raw.csv'
data = pd.read_csv(data_url+fname)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152 entries, 0 to 2151
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   text                  2152 non-null   object
 1   scrape_time           2152 non-null   object
 2   tweet_id              2152 non-null   int64 
 3   created_at            2152 non-null   object
 4   reply_count           2152 non-null   int64 
 5   quote_count           2152 non-null   int64 
 6   like_count            2152 non-null   int64 
 7   retweet_count         2152 non-null   int64 
 8   geo_full_name         50 non-null     object
 9   geo_id                50 non-null     object
 10  geo_bbox              50 non-null     object
 11  tweet_coordinate      6 non-null      object
 12  username              2152 non-null   object
 13  num_followers         2152 non-null   int64 
 14  search_keywords       2152 non-null   object
 15  search_neighbourhood  2152 non-null   

In [6]:
data.head()

,text,scrape_time,tweet_id,created_at,reply_count,quote_count,like_count,retweet_count,geo_full_name,geo_id,geo_bbox,tweet_coordinate,username,num_followers,search_keywords,search_neighbourhood
0,@sarahpottsvic this is true. I live beside Bea...,2022-04-27 11:00:56.487756,1517540279321980929,2022-04-22 16:26:11+00:00,0,0,3,0,NaN,NaN,NaN,NaN,PeggyE2021,10,beacon hill park (bc housing OR vreb OR umbrel...,beacon hill park
1,"TODAY at 12pm: Join PTPG, #TransitIsEssentialM...",2022-04-27 11:00:57.712435,1518566329590161408,2022-04-25 12:23:21+00:00,0,0,17,4,NaN,NaN,NaN,NaN,GreenRootsEJ,2405,beacon hill park (island health OR peers victo...,beacon hill park
2,@sarahpottsvic Very true. I was profoundly imp...,2022-04-27 11:00:59.003974,1517523968898441221,2022-04-22 15:21:23+00:00,2,0,24,5,NaN,NaN,NaN,NaN,MS_athlete,1681,beacon hill park (the aboriginal coalition to ...,beacon hill park
3,The request for witnesses to the crime led to ...,2022-04-27 11:00:59.003974,1517228240116539393,2022-04-21 19:46:15+00:00,4,0,12,1,NaN,NaN,NaN,NaN,Adam_Stirling,9683,beacon hill park (the aboriginal coalition to ...,beacon hill park
4,MA needs affordable and equitable transit! Joi...,2022-04-27 11:00:59.003974,1517228100186263553,2022-04-21 19:45:42+00:00,0,0,1,0,NaN,NaN,NaN,NaN,GBLCBoston,3692,beacon hill park (the aboriginal coalition to ...,beacon hill park


In [7]:
from transformers import pipeline
model = pipeline("sentiment-analysis", model='cardiffnlp/twitter-roberta-base-sentiment-latest', device=0)

Downloading:   0%|          | 0.00/929 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [8]:
model.tokenizer.model_max_length = 512

In [9]:
model('I love GVCEH!')

[{'label': 'Positive', 'score': 0.9876995086669922}]

In [10]:
all_res = []
for res in model(data.text.to_list(), batch_size=32, truncation=True):
  all_res.append(res)
len(all_res)

2152

In [11]:
all_sentiments = [x['label'] for x in all_res]
all_scores = [x['score'] for x in all_res]

In [12]:
data.text.iloc[5], all_sentiments[5], all_scores[5]

('@sarahpottsvic this is true. I live beside Beacon Hill Park and engaged in a one-way dialogue with all city councillors, including yourself, explaining both the impact on me and my family of the tent city council set up, and also offering some different solutions.',
 'Neutral',
 0.7174667119979858)

In [13]:
data['sentiment'] = all_sentiments
data['score'] = all_scores
data.head()

,text,scrape_time,tweet_id,created_at,reply_count,quote_count,like_count,retweet_count,geo_full_name,geo_id,geo_bbox,tweet_coordinate,username,num_followers,search_keywords,search_neighbourhood,sentiment,score
0,@sarahpottsvic this is true. I live beside Bea...,2022-04-27 11:00:56.487756,1517540279321980929,2022-04-22 16:26:11+00:00,0,0,3,0,NaN,NaN,NaN,NaN,PeggyE2021,10,beacon hill park (bc housing OR vreb OR umbrel...,beacon hill park,Neutral,0.717467
1,"TODAY at 12pm: Join PTPG, #TransitIsEssentialM...",2022-04-27 11:00:57.712435,1518566329590161408,2022-04-25 12:23:21+00:00,0,0,17,4,NaN,NaN,NaN,NaN,GreenRootsEJ,2405,beacon hill park (island health OR peers victo...,beacon hill park,Positive,0.699413
2,@sarahpottsvic Very true. I was profoundly imp...,2022-04-27 11:00:59.003974,1517523968898441221,2022-04-22 15:21:23+00:00,2,0,24,5,NaN,NaN,NaN,NaN,MS_athlete,1681,beacon hill park (the aboriginal coalition to ...,beacon hill park,Negative,0.922653
3,The request for witnesses to the crime led to ...,2022-04-27 11:00:59.003974,1517228240116539393,2022-04-21 19:46:15+00:00,4,0,12,1,NaN,NaN,NaN,NaN,Adam_Stirling,9683,beacon hill park (the aboriginal coalition to ...,beacon hill park,Negative,0.727098
4,MA needs affordable and equitable transit! Joi...,2022-04-27 11:00:59.003974,1517228100186263553,2022-04-21 19:45:42+00:00,0,0,1,0,NaN,NaN,NaN,NaN,GBLCBoston,3692,beacon hill park (the aboriginal coalition to ...,beacon hill park,Positive,0.614127


In [16]:
data.to_csv(fname.split('.')[0]+'-sentiment.csv', index=False)